In [ ]:
from github import Github
import os # Used for safer token handling

# --- Best Practice: Load token from an environment variable ---
# In your terminal (before starting jupyter):
# export GITHUB_TOKEN="your_paste_your_token_here"
#
# Then in Python:
# GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

# --- For this example, we'll assign it directly (less secure) ---
GITHUB_TOKEN = "Your token" # <--- PASTE YOUR GITHUB TOKEN HERE

# Login to GitHub
g = Github(GITHUB_TOKEN)

# Get the authenticated user
user = g.get_user()
print(f"Successfully logged in as: {user.login}")

/tmp/ipykernel_10062/717409637.py:15: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(GITHUB_TOKEN)


Successfully logged in as: azario0


# List All Your Repositories

In [2]:
print("Listing all accessible repositories:")
all_repos = user.get_repos()

for repo in all_repos:
    print(f"- {repo.full_name}")

Listing all accessible repositories:
- azario0/100-days-py
- azario0/100DaysOfCode
- azario0/101-rs
- azario0/11
- azario0/17-project
- azario0/advanced-screen-recorder
- azario0/advanced_rag
- azario0/advanced_rag_game
- azario0/age-group-prediction
- azario0/ai-grammar-corrector
- azario0/AI-Recipe-Generator
- azario0/algo
- azario0/Algorithms
- azario0/Alphabeta-minmax
- azario0/analyse-boulangerie
- azario0/analyse-de-mots-de-pass
- azario0/analyse-de-phrase
- azario0/Analyse-emmision-co2
- azario0/analyse-kaggle
- azario0/analyse-sleep
- azario0/analysedatasetpython
- azario0/analyse_app_predict
- azario0/analyse_pizzeria
- azario0/analytics
- azario0/Animagine-XL-4.0
- azario0/animal-classification-0
- azario0/annualenterprisesurver2021
- azario0/app-reservation-hotel
- azario0/app-stabilisateur-video
- azario0/app-to-store-manager
- azario0/application-de-gestion-d-etudiants
- azario0/apprentissage_symbolique_SVM
- azario0/app_boulangerie
- azario0/arcade-game
- azario0/Arduino_

# Select and List the Contents of a Specific Repo

In [5]:
repo_full_name = "azario0/flask-api-dictionnary" 

try:
    # Get the repository object
    my_repo = g.get_repo(repo_full_name)
    print(f"Successfully selected repo: '{my_repo.full_name}'")
    
    # List contents of the root directory
    # The empty string "" means the root of the repo
    contents = my_repo.get_contents("")
    
    print("\nContents of the repository root:")
    for content_file in contents:
        # Check if it's a directory or a file
        item_type = "Dir " if content_file.type == "dir" else "File"
        print(f"- [{item_type}] {content_file.path}")

except Exception as e:
    print(f"Error: Could not find repo '{repo_full_name}'. Is the name correct? Details: {e}")

Successfully selected repo: 'azario0/flask-api-dictionnary'

Contents of the repository root:
- [File] README.md
- [File] app.py
- [File] notebook.ipynb


# Download a Repository (as a .zip file)

In [6]:
import requests

# Use the 'my_repo' object from the previous cell
# Or get it again if you're running this cell independently
# repo_full_name = "your_username/your_repo_name" 
# my_repo = g.get_repo(repo_full_name)

# Get the archive link (zip format by default)
zip_url = my_repo.get_archive_link("zipball")
print(f"Archive URL: {zip_url}")

# Download the file
response = requests.get(zip_url, allow_redirects=True)

# Save the downloaded zip file
zip_filename = f"{my_repo.name}.zip"
with open(zip_filename, 'wb') as f:
    f.write(response.content)

print(f"Repository '{my_repo.full_name}' successfully downloaded as '{zip_filename}'")

Archive URL: https://codeload.github.com/azario0/flask-api-dictionnary/legacy.zip/refs/heads/main
Repository 'azario0/flask-api-dictionnary' successfully downloaded as 'flask-api-dictionnary.zip'


# Create a New Repository

In [7]:
# --- Example 1: Create a simple, empty private repository ---
try:
    new_repo_simple = user.create_repo(
        name="my-awesome-api-repo",
        description="A repo created via the GitHub API",
        private=True
    )
    print(f"Successfully created simple repo: {new_repo_simple.html_url}")
except Exception as e:
    print(f"Error creating simple repo (it might already exist): {e}")


# --- Example 2: Create a public repo with a README, .gitignore, and MIT License ---
try:
    new_repo_advanced = user.create_repo(
        name="my-public-project",
        description="A more complete repo created via the API",
        private=True,                # This could be False 
        auto_init=True,              # This creates a README.md
        gitignore_template="Python", # Specify a gitignore template (e.g., "Python", "Node")
        license_template="mit"       # Specify a license (e.g., "mit", "gpl-3.0")
    )
    print(f"Successfully created advanced repo: {new_repo_advanced.html_url}")
except Exception as e:
    print(f"Error creating advanced repo (it might already exist): {e}")

Successfully created simple repo: https://github.com/azario0/my-awesome-api-repo
Successfully created advanced repo: https://github.com/azario0/my-public-project


# Select a Repo and Delete a File or Folder

In [8]:
# Select the repository you want to modify
repo_to_modify_name = "azario0/my-public-project" # Use a repo you just created or another test repo
repo = g.get_repo(repo_to_modify_name)

# Specify the path to the file you want to delete
file_path_to_delete = "README.md"
commit_message = "feat: remove default README file"

try:
    # Get the file's contents object
    file_contents = repo.get_contents(file_path_to_delete)
    
    # Delete the file
    delete_result = repo.delete_file(
        path=file_contents.path,
        message=commit_message,
        sha=file_contents.sha
    )
    print(f"Successfully deleted file '{file_path_to_delete}'.")
    print(f"Commit SHA: {delete_result['commit'].sha}")

except Exception as e:
    print(f"Error deleting file '{file_path_to_delete}': {e}")

Successfully deleted file 'README.md'.
Commit SHA: 103a395bbf4b29a823f85d586c7349aad7c387b4


# Deleting a repo

In [10]:
from github.GithubException import UnknownObjectException

# --- CONFIGURATION ---
# The full name of the repository you want to PERMANENTLY DELETE.
# Example: "my-username/my-test-repo-to-delete"
REPO_TO_DELETE = "azario0/my-awesome-api-repo" 

# --- SCRIPT ---
try:
    # Get the repository object
    repo = g.get_repo(REPO_TO_DELETE)
    
    # The actual delete command
    repo.delete()
    
    print(f"✅ Successfully deleted repository: {REPO_TO_DELETE}")

except UnknownObjectException:
    print(f"❌ Error: Repository '{REPO_TO_DELETE}' not found. Check the name and your permissions.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

✅ Successfully deleted repository: azario0/my-awesome-api-repo


# How to Delete a File

In [11]:
# --- CONFIGURATION ---
REPO_NAME = "azario0/my-public-project"
FILE_TO_DELETE = ".gitignore"  # e.g., "README.md" or "data/old_data.csv"
COMMIT_MESSAGE = "chore: remove obsolete file via API"

# --- SCRIPT ---
try:
    # Get the repository object
    repo = g.get_repo(REPO_NAME)
    
    # Get the file's contents object to access its SHA hash
    file_contents = repo.get_contents(FILE_TO_DELETE)
    
    # Delete the file using its path, a commit message, and its SHA
    delete_result = repo.delete_file(
        path=file_contents.path,
        message=COMMIT_MESSAGE,
        sha=file_contents.sha
    )
    
    print(f"✅ Successfully deleted file '{FILE_TO_DELETE}'")
    print(f"   Commit SHA: {delete_result['commit'].sha}")

except UnknownObjectException:
    print(f"❌ Error: Could not find file '{FILE_TO_DELETE}' in repo '{REPO_NAME}'. Check the path.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

✅ Successfully deleted file '.gitignore'
   Commit SHA: d1ede19a32cda4055b17fc3b7ac6ed4d14b9c907


# How to Delete a folder

In [16]:
def delete_folder_contents(repo, folder_path):
    """
    Recursively deletes all files and sub-folders within a given folder path.
    """
    try:
        contents = repo.get_contents(folder_path)
        
        for item in contents:
            if item.type == "dir":
                # If it's a directory, recurse into it
                print(f"   Entering sub-directory: {item.path}")
                delete_folder_contents(repo, item.path)
            else:
                # If it's a file, delete it
                print(f"   - Deleting file: {item.path}")
                repo.delete_file(
                    path=item.path,
                    message=f"chore: remove {item.name}",
                    sha=item.sha
                )
        
        print(f"✅ Finished processing folder: {folder_path}")

    except UnknownObjectException:
        print(f"⚠️ Warning: Folder '{folder_path}' not found or is already empty. Skipping.")
    except Exception as e:
        print(f"❌ An error occurred processing '{folder_path}': {e}")
        # Re-raise the exception to stop the process if a critical error occurs
        raise e

# --- CONFIGURATION ---
REPO_NAME = "azario0/my-public-project"
FOLDER_TO_DELETE = "greetings" # The path to the folder you want to empty

# --- SCRIPT ---
try:
    print(f"Starting deletion process for folder '{FOLDER_TO_DELETE}' in repo '{REPO_NAME}'...")
    repo = g.get_repo(REPO_NAME)
    delete_folder_contents(repo, FOLDER_TO_DELETE)
    print("\n🎉 Deletion process complete.")
except Exception as e:
    print(f"\n❌ The script stopped due to an error: {e}")

Starting deletion process for folder 'greetings' in repo 'azario0/my-public-project'...
   - Deleting file: greetings/api_hello.txt
✅ Finished processing folder: greetings

🎉 Deletion process complete.


# Upload a file 

In [13]:
# Create a dummy file to upload
local_file_name = "hello_world.txt"
with open(local_file_name, "w") as f:
    f.write("Hello from my Jupyter Notebook!\n")
    f.write("This file was uploaded via the GitHub API.")

print(f"📄 Created local file: '{local_file_name}'")

📄 Created local file: 'hello_world.txt'


In [17]:
from github.GithubException import GithubException, UnknownObjectException

# --- CONFIGURATION ---
REPO_NAME = "azario0/my-public-project"
LOCAL_FILE_PATH = "hello_world.txt" # The file we just created
REMOTE_FILE_PATH = "greetings/api_hello.txt" # Path inside the repo (folders will be created)
COMMIT_MESSAGE = "feat: add greeting file via API"

# --- SCRIPT ---
try:
    # Get the repository object
    repo = g.get_repo(REPO_NAME)
    
    # Read the content of the local file
    with open(LOCAL_FILE_PATH, 'r') as f:
        content = f.read()
    
    # Upload the file to the repository
    # The create_file method handles encoding for you.
    repo.create_file(
        path=REMOTE_FILE_PATH,
        message=COMMIT_MESSAGE,
        content=content
    )
    
    print(f"✅ Successfully uploaded '{LOCAL_FILE_PATH}' to '{REMOTE_FILE_PATH}' in repo '{REPO_NAME}'.")

except UnknownObjectException:
    print(f"❌ Error: Repository '{REPO_NAME}' not found.")
except GithubException as e:
    # This often happens if the file already exists.
    if e.status == 422:
        print(f"❌ Error: File '{REMOTE_FILE_PATH}' already exists in the repository.")
        print("   Consider using the 'update_file()' method if you want to modify it.")
    else:
        print(f"❌ An unexpected GitHub error occurred: {e}")
except FileNotFoundError:
    print(f"❌ Error: The local file '{LOCAL_FILE_PATH}' was not found.")

✅ Successfully uploaded 'hello_world.txt' to 'greetings/api_hello.txt' in repo 'azario0/my-public-project'.


# How to Upload a Folder

In [18]:
# Create a dummy folder structure
os.makedirs("my_local_project/data", exist_ok=True)

with open("my_local_project/main.py", "w") as f:
    f.write("# Main Python script\nprint('Hello Project!')")
    
with open("my_local_project/README.md", "w") as f:
    f.write("# My Awesome Project\nThis project was uploaded via script.")

with open("my_local_project/data/results.csv", "w") as f:
    f.write("id,value\n1,100\n2,150")

print("📂 Created local folder structure: 'my_local_project'")

📂 Created local folder structure: 'my_local_project'


In [19]:
# --- CONFIGURATION ---
REPO_NAME = "azario0/my-public-project"
LOCAL_FOLDER_PATH = "my_local_project"
# The base path in the repo where the folder content will be uploaded.
# Use "" for the root, or e.g., "src/" to place it in a 'src' folder.
REMOTE_FOLDER_PATH = "project_from_api" 

# --- SCRIPT ---
try:
    repo = g.get_repo(REPO_NAME)
    
    # Walk through the local folder
    for root, dirs, files in os.walk(LOCAL_FOLDER_PATH):
        for file in files:
            local_path = os.path.join(root, file)
            
            # Construct the remote path
            # We need to remove the local folder base from the path
            relative_path = os.path.relpath(local_path, LOCAL_FOLDER_PATH)
            # On Windows, os.path uses backslashes, but GitHub API needs forward slashes
            remote_path = os.path.join(REMOTE_FOLDER_PATH, relative_path).replace('\\', '/')

            try:
                with open(local_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                # Check if file already exists
                try:
                    repo.get_contents(remote_path)
                    print(f"   - ⚠️ Skipping '{remote_path}', file already exists.")
                    continue # Skip to the next file
                except UnknownObjectException:
                    # File does not exist, so we can create it
                    pass

                # Upload the file
                repo.create_file(
                    path=remote_path,
                    message=f"feat: add {file}",
                    content=content
                )
                print(f"   - ✅ Uploaded '{local_path}' to '{remote_path}'")
            
            except Exception as e:
                # Catch potential encoding errors for non-text files
                print(f"   - ❌ Failed to upload '{local_path}': {e}")

    print("\n🎉 Folder upload process complete.")

except UnknownObjectException:
    print(f"❌ Error: Repository '{REPO_NAME}' not found.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

   - ✅ Uploaded 'my_local_project/README.md' to 'project_from_api/README.md'
   - ✅ Uploaded 'my_local_project/main.py' to 'project_from_api/main.py'
   - ✅ Uploaded 'my_local_project/data/results.csv' to 'project_from_api/data/results.csv'

🎉 Folder upload process complete.
